# 瞬态热力学分析 {#ref_thermal_transient}

此示例演示如何了使用 PyMAPDL
输入随时间变化的温度表，以改变梁的温度。它使用对流系数和体积温度独立变化的对流载荷。

示例改编自：
<https://www.simutechgroup.com/tips-and-tricks/fea-articles/97-fea-tips-tricks-thermal-transient>

Thanks SimuTech!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from ansys.mapdl.core import launch_mapdl

mapdl = launch_mapdl(loglevel="ERROR")

mapdl.clear()
mapdl.prep7()

# 材料性能 -- 英制 1020 钢
mapdl.units("BIN")  # U.S. Customary system using inches (in, lbf*s2/in, s, °F).
mapdl.mp("EX", 1, 30023280.0)
mapdl.mp("NUXY", 1, 0.290000000)
mapdl.mp("ALPX", 1, 8.388888889e-06)
mapdl.mp("DENS", 1, 7.346344000e-04)
mapdl.mp("KXX", 1, 6.252196000e-04)
mapdl.mp("C", 1, 38.6334760)

# 使用热单元类型
mapdl.et(1, "SOLID70")

# Geometry and Mesh

创建一个大小为 5x1x1 英寸的块并对其进行网格划分


In [ ]:
mapdl.block(0, 5, 0, 1, 0, 1)
mapdl.lesize("ALL", 0.2, layer1=1)

mapdl.mshape(0, "3D")
mapdl.mshkey(1)
mapdl.vmesh(1)
mapdl.eplot()

# Setup the Solution

求解瞬态分析，同时增加和减少荷载。

请注意下段代码片段中的求解时间命令。最终 TIME 设置为 1000 秒。在 DELTIM
命令中，时间子步的大小允许从最少 2 秒到最多 50 秒不等。 第一个子步为 10
秒。自动时间步的大小将在两个极端之间改变。

与时间相关的对流系数值使用表格数组。时间放在第零列，相关的对流系数放在第一列。


In [ ]:
mapdl.run("/SOLU")
mapdl.antype(4)  # 瞬态分析
mapdl.trnopt("FULL")  # 全瞬态分析
mapdl.kbc(0)  # ramp loads up and down

# 时间步
end_time = 1500
mapdl.time(end_time)  # end time for load step
mapdl.autots("ON")  # use automatic time stepping


# 设置，其中 subset 时间为 10 秒，时间
mapdl.deltim(10, 2, 25)  # 子步长（秒）
#                          -- 小于下表数组中最小时间变化的最小值

# 创建对流时间和系数表，并将其转入 MAPDL
my_conv = np.array(
    [
        [0, 0.001],  # start time
        [120, 0.001],  # end of first "flat" zone
        [130, 0.005],  # ramps up in 10 seconds
        [700, 0.005],  # end of second "flat zone
        [710, 0.002],  # ramps down in 10 seconds
        [end_time, 0.002],
    ]
)  # end of third "flat" zone
mapdl.load_table("my_conv", my_conv, "TIME")


# Create a table of bulk temperatures for a given time and transfer to MAPDL
my_bulk = np.array(
    [
        [0, 100],  # start time
        [120, 100],  # end of first "flat" zone
        [500, 300],  # ramps up in 380 seconds
        [700, 300],  # hold temperature for 200 seconds
        [900, 75],  # temperature ramps down for 200 seconds
        [end_time, 75],
    ]
)  # end of second "flat" zone
mapdl.load_table("my_bulk", my_bulk, "TIME")

# The Transient Thermal Solve

该模型将在一个时间步长内求解。因此，需要使用 `TSRES`
命令强制求解器在上述两个表阵列的每个时间点都包含一个 `SOLVE` 。
这样可以确保瞬态分析遵循随时间变化的曲线。在 `TSRES`
时间点之间的中间解将根据 `DELTIM` 命令和 ANSYS
求解器的自动时间步进决定包含在内。

在本例中，上图中的 `TSRES`
数组的时间是手动确定的。在更复杂的建模情况下，可以使用一组 APDL
命令对选定的表数组条目自动执行这一过程，包括检查时间间隔是否过短。

Results at substeps will be wanted if the intermediate solutions of the
time-transient analysis are to be available for post-processing review.
The `OUTRES` command is used to control how much is written to the
results file. In this example the OUTRES command will be used to simply
write out all results for all substeps. In work with large models and
may substeps, too much data will be written if such a strategy is
employed for `OUTRES`, and other options will need to be considered.
Note that one option for the `OUTRES` command is to control times at
which results are written with a Table Array, much as is used in the
`TSRES` command, but typically for a larger number of time points,
although including those of the TSRES array.

The initial condition starting temperature is controlled for this
example with the `TUNIF` command. Note that thermal transient analyses
can also have a starting temperature profile formed by a static thermal
`SOLVE`. If a user neglects to set an initial temperature in ANSYS
Mechanical APDL, a value of zero will be used, which is often not what
is desired.

The thermal convective loads are applied with an SF family command---in
this example a convective load is applied to the end face of the solid
model by the SFA command, using the Table Array entries for convection
and bulk temperature that were developed above. The Table Array names
are surrounded with percent signs (%). A SOLVE is then performed.


In [ ]:
# Force transient solve to include the times within the conv and bulk arrays
# my_tres = np.unique(np.vstack((my_bulk[:, 0], my_conv[:, 0])))[0]  # same as
mapdl.parameters["my_tsres"] = [120, 130, 500, 700, 710, 900, end_time]
mapdl.tsres("%my_tsres%")

mapdl.outres("ERASE")
mapdl.outres("ALL", "ALL")

mapdl.eqslv("SPARSE")  # use sparse solver
mapdl.tunif(75)  # force uniform starting temperature (otherwise zero)

# apply the convective load (convection coefficient plus bulk temperature)
# use "%" around table array names
mapdl.sfa(6, 1, "CONV", "%my_conv%", " %my_bulk%")

# solve
mapdl.solve()

# Post-Processing

Open up the result file using `ansys.mapdl.reader` result =
mapdl.thermal_result


In [ ]:
mapdl.post1()

# Visualize a Slice

Visualize a slice through the dataset using `pyvista` for more details
visit [pyvista documentation]().


In [ ]:
# get the temperature of the 30th result set
mapdl.set(1, 30)
temp = mapdl.post_processing.nodal_temperature()

# Load this result into the underlying VTK grid
grid = mapdl.mesh._grid
grid["temperature"] = temp

# generate a single horizontal slice slice along the XY plane
single_slice = grid.slice(normal=[0, 0, 1], origin=[0, 0, 0.5])
single_slice.plot(scalars="temperature")

# Visualize Several Slices

This shows how you can visualize a series of slices through a dataset


In [ ]:
# get the temperature of a different result set
mapdl.set(1, 120)
temp = mapdl.post_processing.nodal_temperature()

# Load this result into the underlying VTK grid
grid = mapdl.mesh._grid
grid["temperature"] = temp

# generate a single horizontal slice slice along the XY plane
slices = grid.slice_along_axis(7, "y")
slices.plot(scalars="temperature", lighting=False, show_edges=True)

# Temperature at a Single Point

Extract the temperature at a single node and plot it with respect to the
input temperatures using `ansys.mapdl`

Here, we use the `get_value` command which is very similar to the `*GET`
command, except it immediately returns the value as a python accessible
variable, rather than storing it to a MAPDL value.


In [ ]:
# for example, the temperature of Node 12 is can be retrieved simply with:
mapdl.get_value("node", 12, "TEMP")

# note that this is similar to # *GET, Par, NODE, N, Item1, IT1NUM, Item2, IT2NUM
# See the MAPDL reference for all the items you can obtain using *GET

Here, we extract the temperature of the node across for each solution


In [ ]:
nsets = mapdl.post_processing.nsets
node_temp = []
for i in range(1, 1 + nsets):
    mapdl.set(1, i)
    node_temp.append(mapdl.get_value("node", 12, "TEMP"))

# here are the first 10 temperatures
node_temp[:10]

Alternatively, you can simply grab the data for the node from the entire
response. This is less efficient as the entire data set is sent back for
each result.


In [ ]:
# get the index of node 12 in MAPDL
idx = np.nonzero(mapdl.mesh.nnum == 12)[0][0]

# get the temperature at that index for each result
node_temp_from_post = []
for i in range(1, 1 + nsets):
    mapdl.set(1, i)
    node_temp_from_post.append(mapdl.post_processing.nodal_temperature()[idx])

# Again, the first 10 temperatures
node_temp_from_post[:10]

Plot the temperature as a function of time


In [ ]:
time_values = mapdl.post_processing.time_values
plt.plot(time_values, node_temp, label="Node 12")
plt.plot(my_bulk[:, 0], my_bulk[:, 1], ":", label="Input")
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Temperature ($^\circ$F)")
plt.show()

# Stop mapdl


In [ ]:
mapdl.exit()